# Old functions


In [ ]:
def get_price_probability(hotel_data, selected_price):
    """Calculate the probability that the competence chose selected_price

    :param dict_prices: Dict with the prices used by the competence
    :param selected_price: Tuple string with the selected prices for the competence
    :returns: Probability that the competence choose the selected_price
    :rtype: Double

    """

    comp_hotel_data = {}
    result = 1

    # We stract the price clients that we have used want from competence data
    for indi, hotel_name in enumerate(LOCAL_COMPETENCE_HOTELS):
        comp_hotel_data[hotel_name] = {}
        comp_hotel_data[hotel_name]['total'] = 0
        comp_hotel_data[hotel_name]['chosen'] = 0

        for index, row in hotel_data[hotel_name].iterrows():
            price = selected_price[indi]
            if index == price:
                comp_hotel_data[hotel_name]['chosen'] += row['clients']
            comp_hotel_data[hotel_name]['total'] += row['clients']

    for hotel_name in comp_hotel_data:
        result *= comp_hotel_data[hotel_name]['chosen'] / comp_hotel_data[hotel_name]['total']

    return result

In [1]:
def generate_ara_values():
    """Generate ARA values from the formulas

    :returns: Array with all of the ARA values
    :rtype: List

    """
    ara_values = {}
    for room_price, comp_prices in prices.items():
        temp_ara_val = 0
        for outcome in [0, 1]:
            for comp_price, our_clients in comp_prices.items():
                if outcome == 1:
                    temp_ara_val += get_utility(room_price, outcome) * \
                        get_outcome_prob(room_price, comp_price, our_clients) * \
                        get_price_probability(comp_prices, comp_price)
                else:
                    temp_ara_val -= get_utility(room_price, outcome) * \
                        (1 - get_outcome_prob(room_price, comp_price, our_clients)) * \
                        get_price_probability(comp_prices, comp_price)
        ara_values[room_price] = temp_ara_val

    return ara_values

In [2]:
def get_price_probability(dict_prices, selected_price):
    """Calculate the probability that the competence chose selected_price

    :param dict_prices: Dict with the prices used by the competence
    :param selected_price: Tuple string with the selected prices for the competence
    :returns: Probability that the competence choose the selected_price
    :rtype: Double

    """

    comp_hotel_data = {}
    result = 1

    chosen_prices = selected_price.split(",")
    # We stract the price clients that we have used want from competence data
    for indi, hotel_name in enumerate(comp_hotels_names):
        comp_hotel_data[hotel_name] = {}
        comp_hotel_data[hotel_name]['total'] = 0

        for tuple_price in dict_prices.keys():
            price = tuple_price.split(",")[indi]
            if chosen_prices[indi] == price:
                comp_hotel_data[hotel_name]['chosen'] = sum(comp_data[hotel_name][price].values())
            comp_hotel_data[hotel_name]['total'] += sum(comp_data[hotel_name][price].values())


    # print(comp_hotel_data)
    for hotel_name in comp_hotel_data:
        result *= comp_hotel_data[hotel_name]['chosen'] / comp_hotel_data[hotel_name]['total']

    return result

In [3]:
def get_outcome_prob(our_price, comp_price, our_clients):
    """Calculate the probability that a client comes to our hotel given our price and competence price

    :param our_price: Self price of the room
    :param comp_price: Competence price for the same room
    :param our_clients: Number of clients that have chosen us in the past
    :returns: Probability of choosing our hotel
    :rtype: Double

    """
    
    total_clients = our_clients
    # Search for competence data to calculate the demand
    for indi, comp_hotel in enumerate(comp_hotels_names):
        if comp_data[comp_hotel]:
            try:
                selected_comp_price = comp_price.split(",")[indi]
                temp_comp_price = comp_price.split(",")
                del temp_comp_price[indi]
                temp_comp_price.insert(0, our_price)
                temp_comp_price = ",".join(temp_comp_price)
                
                total_clients += comp_data[comp_hotel][selected_comp_price][temp_comp_price]
            # If the price is not in the competences prices list
            except Exception as e:
                print("Precio no encontrado en la competencia")
                total_clients += our_clients
        # If we dont have competence data, we asume they have the same clients than us
        else:
            print("Esto significa que hay algo mal")
            total_clients += our_clients

    return our_clients / total_clients